# Grep demo

In [1]:
import iknowpy

iknow = iknowpy.iKnowEngine()

In [34]:
iknow.index("The patient did not report kidney beans, but had a lower back pain after gardening on Saturday.","en")
# iknow.index("He wasn't very happy with the cake, even though we put much effort (and no sugar) into it.","en")

In [8]:
import pprint
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(iknow.m_index)

{ 'proximity': [ ((3, 5), 64),
                 ((9, 11), 64),
                 ((5, 9), 42),
                 ((3, 9), 32),
                 ((5, 11), 32),
                 ((3, 11), 25)],
  'sentences': [ { 'entities': [ { 'dominance_value': 0.0,
                                   'entity_id': 1,
                                   'index': 'he',
                                   'offset_start': 0,
                                   'offset_stop': 2,
                                   'type': 'PathRelevant'},
                                 { 'dominance_value': 500.0,
                                   'entity_id': 2,
                                   'index': "wasn't",
                                   'offset_start': 3,
                                   'offset_stop': 9,
                                   'type': 'Relation'},
                                 { 'dominance_value': 1000.0,
                                   'entity_id': 3,
                                   'index

In [43]:
# print basic parsing output
for s in iknow.m_index['sentences']:
    for e in s['entities']:
        print('<'+e['type']+'>'+e['index']+'</'+e['type']+'>', end=' ')
    print('\n')

<NonRelevant>the</NonRelevant> <Concept>patient</Concept> <Relation>did not report</Relation> <Concept>kidney beans</Concept> <Relation>but had</Relation> <NonRelevant>a</NonRelevant> <Concept>lower back pain</Concept> <Relation>after gardening on</Relation> <Concept>saturday</Concept> 



In [42]:
# annotate negated entities
for s in iknow.m_index['sentences']:
    
    # first build an array marking start and end positions
    # position info relates to positions within the path, so exclude NonRelevant entities
    pos = 0
    negated = []
    for a in s['path_attributes']:
        if a['type']=="Negation":
            print('attribute: '+str(a))
            for p in range(pos,a['pos']-1):  # append zeroes since last attribute
                negated.append(0)
            for p in range(0,a['span']):     # append zeroes for this span
                negated.append(1)
            pos = a['pos']+a['span']-1       # update position

    for p in range(pos, len(s['path'])-1):
        negated.append(0)
        
    print('\nnegated path positions: '+str(negated)+'\n')
    
    pos = 0
    for e in s['entities']:
        
        # ignore NonRelevants
        if e['type'] == 'NonRelevant': 
            continue
        
        if negated[pos]: print("<neg>", end='')
        
        print(e['index'], end='')
        
        if negated[pos]: print("</neg>", end='')
        
        print(" ", end="")
        pos = pos+1
    print('\n')

attribute: {'type': 'Negation', 'pos': 0, 'span': 3}

negated path positions: [1, 1, 1, 0, 0, 0, 0]

<neg>patient</neg> <neg>did not report</neg> <neg>kidney beans</neg> but had lower back pain after gardening on saturday 



In [56]:
# now turn into a function that relates back to the original text
def strip_negative(text, language="en", iknow=iknowpy.iKnowEngine()):
    iknow.index(text,language)
    stripped = ""
    for s in iknow.m_index['sentences']:
        pos = 0
        negated = []
        for a in s['path_attributes']:
            if a['type']=="Negation":
                for p in range(pos,a['pos']-1):  # append zeroes since last attribute
                    negated.append(0)
                for p in range(0,a['span']):     # append zeroes for this span
                    negated.append(1)
                pos = a['pos']+a['span']-1       # update position
        for p in range(pos, len(s['path'])-1):
            negated.append(0)
        pos = 0
        for e in s['entities']:
            if e['type'] == 'NonRelevant':
                continue
            if negated[pos] != 1: 
                stripped += text[e['offset_start']:e['offset_stop']] + " "
            pos = pos+1
        stripped += '\n'
    return stripped
        

print(strip_negative("He wasn't very happy with the cake, even though we put much effort (no sugar) into it."))

even though we put much effort into it. 

